<a href="https://colab.research.google.com/github/tuhinkm/Mathematical_Optimization/blob/main/Constrained_TSP_with_heuristic_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TSP using GA or ACO are not an old problem, rather it is one of the oldest problem and solution combination. Any heuristic is typically good for quick and rough solution generation however in real-life, business constraints are crucial inputs that one should consider while generating the solution. Adding constraints to TSP will reduce the search space and in turn the time, however the challenge lies how to add such conditions in GA or ACO format. In this implmentation, we are creating a framework to design GA with constraints to solve TSP

In [1]:
# import block
import random
import numpy as np

In [120]:
###--- business conditions and validations ---###

# First city must not be a specific city
def first_city_not(chromosome, city_number):
  '''
  check and validate that first city to travel from source is not a specific city. In thant case the validation function will return 0, else 1
  '''
  if chromosome[0] == city_number:
    # print(f"{chromosome} is not valid because it starts with city number {city_number} which is not supposed to happend")
    return 0
  else:
    return 1

# Last city must not be a specific city
def last_city_not(chromosome, city_number):
  '''
  check and validate that last city to travel before source is not a specific city. In thant case the validation function will return 0, else 1
  '''
  if chromosome[-1] == city_number:
    # print(f"{chromosome} is not valid because it ends with city number {city_number} which is not supposed to happend")
    return 0
  else:
    return 1

# If travel to a specific city then must visit a secific city next
def concurrent_city_visit(chromosome, city_sequence_visit):
  '''
  check and validate that last city to travel before source is not a specific city. In thant case the validation function will return 0, else 1
  '''
  chromosome_list =  ' '.join(map(str, chromosome))
  sequence_list = ' '.join(map(str, city_sequence_visit))
  if sequence_list in chromosome_list:
    return 1
  else:
    # print(f"{chromosome} is not valid because it does not contain the city travel sequence {city_sequence_visit}")
    return 0

# If travel to a specific city then must avoid a sepecific city next
def concurrent_city_avoid(chromosome, city_sequence_visit, city_sequence_avoid):
  '''
  check and validate that last city to travel before source is not a specific city. In thant case the validation function will return 0, else 1
  '''
  chromosome_list =  ' '.join(map(str, chromosome))
  sequence_visit_list = ' '.join(map(str, city_sequence_visit))
  sequence_avoid_list = ' '.join(map(str, city_sequence_avoid))

  if (sequence_visit_list+' '+sequence_avoid_list) in chromosome_list:
    # print(f"{chromosome} is not valid because it contains the city travel sequence {city_sequence_visit} and {city_sequence_avoid}")
    return 0
  else:
    return 1

# overall validation function

def overall_validation(chromosome, first_city, last_city, city_sequence_visit, city_sequence_avoid):
  '''
  It checks overall conditions and provides a final go-ahead conclusion
  '''
  check1 = first_city_not(chromosome, first_city)
  check2 = last_city_not(chromosome, last_city)
  check3 = concurrent_city_visit(chromosome, city_sequence_visit)
  check4 = concurrent_city_avoid(chromosome, city_sequence_visit, city_sequence_avoid)
  return (check1*check2*check3*check4)

In [121]:
# initialization
def initial_population_generation(node_number, population_size, starting_node_id, first_city, last_city, city_sequence_visit, city_sequence_avoid):
  '''
  This function generates initial population of chromosome. While they are being generated, we can call the validation function that time or during mutation/crossover. I have chosen the second option.
  '''

  potential_solution = [i for i in range(1, node_number+1) if i!=starting_node_id] # excluding start node
  population = []

  for i in range(population_size):
      random.shuffle(potential_solution)
      solution_validation = overall_validation(potential_solution, first_city, last_city, city_sequence_visit, city_sequence_avoid)
      while(solution_validation==0):
          print(f"solution {potential_solution} is invalid, regenerating chromosome...")
          random.shuffle(potential_solution)
          solution_validation = overall_validation(potential_solution, first_city, last_city, city_sequence_visit, city_sequence_avoid)
      # print(f"valid solution {potential_solution} found, adding to pool...")
      population.append(potential_solution.copy())

  return population

# fitness function calculation
def fitness_calculation(population,time_matrix,starting_node_id):
    '''
    This function will calculate fitness of a population chromosome by chromosome. In TSP's terminology, it will calculate cost of travel for a potenrial solution
    '''
    fitness_dictionary = {}
    for chromosome in population:
        cost=0
        for i in range(len(chromosome)):
            if i==0:
              # print(f"For {i} position, the cost to be added is {time_matrix[starting_node_id-1][chromosome[i]-1]}")
              cost+=time_matrix[starting_node_id-1][chromosome[i]-1]
            elif i==len(chromosome)-1:
              # print(f"For {i} position, the cost to be added is {time_matrix[chromosome[i]-1][starting_node_id-1]}")
              cost+=time_matrix[chromosome[i]-1][starting_node_id-1]
            else:
              # print(f"For {i} position, the cost to be added is {time_matrix[chromosome[i-1]-1][chromosome[i]-1]}")
              cost+=time_matrix[chromosome[i-1]-1][chromosome[i]-1]
        fitness_dictionary[tuple(chromosome)] = round(10000/cost,2)

    return (fitness_dictionary)

In [134]:
'''
For new generation, we shall consider some of the population will go directly without any modification in chromosome. Some other part of the population will go for crossover, mutation or both. In this example, we are considering the scenario as below:
parent to next generation - top 10%
crossover - top 85%
mutation - 5% randomly from crossover mutation
bottom 5% - disqualified
'''
def pool_division(population_dictionary):
  '''
  This function bucketizes the list of population for next generation. It will return the buckets as dictionary format
  '''
  population_size = len(population_dictionary.keys())
  top_10_percent = int(population_size*0.1)
  top_85_percent = int(population_size*0.85)
  sorted_population = sorted(population_dictionary.items(), key=lambda x: x[1], reverse=True)

  # Extract the top populations
  top_10_percent_population = [list(route) for route in dict(sorted_population[:top_10_percent]).keys()]
  top_85_percent_population = [list(route) for route in dict(sorted_population[:top_85_percent]).keys()]

  return [top_10_percent_population, top_85_percent_population]

### Crossover and mutation functions

# offspring through crossover
def ordered_crossover(p1, p2, start, end):
  '''
  This function creates an ordered crossover between two parents
  '''
  # Initialize offspring with None values
  length = len(p1)
  offspring = [None] * length

  # Copy the segment from parent1
  offspring[start:end] = p1[start:end]

  # Fill remaining positions with genes from parent2 in order
  p2_genes = [gene for gene in p2 if gene not in offspring[start:end]]

  # Fill positions before the copied segment
  for i in range(start):
      offspring[i] = p2_genes.pop(0)

  # Fill positions after the copied segment
  for i in range(end, length):
      offspring[i] = p2_genes.pop(0)

  return offspring

def random_mutation(chromosome):
  '''
  Swap positions of two genes randomly in a chromosome. Perform the random swap 20% time of lengh of chromosome. This will ensure enough mutation happened in the population and all conditions are validated by the new chromosome
  '''
  for i in range(int(len(chromosome)*0.2)):
  # Randomly select two different indices
    indices = random.sample(range(len(chromosome)),2)
    idx1, idx2 = indices[0], indices[1]
    chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]

  return(chromosome)

# valid child generation
def valid_child_generation(parent1,parent2, first_city, last_city, city_sequence_visit, city_sequence_avoid):
  '''
  This function will take two parent chromosome and perform random ordered crossover. Post child generation, it will validate if the childs are valid. If not, then there are two way possible. The offspring function can be called repeatedlly unless valid child is generated. Else, random mutation in the child can be done so that the child is valid. In this case, I shall take first option in case both the children are not vaild. If one is valid, we shall follow second method for other child till it becomes valid
  '''
  child1=[]
  child2=[]

  length = len(parent1)
  # Choose two crossover points
  start = random.randint(0, length - 2)
  end = random.randint(start + 1, length)

  child1 = ordered_crossover(parent1, parent2, start, end)
  child2 = ordered_crossover(parent2, parent1, start, end)

  child1_validation = overall_validation(child1, first_city, last_city, city_sequence_visit, city_sequence_avoid)
  child2_validation = overall_validation(child2, first_city, last_city, city_sequence_visit, city_sequence_avoid)

  # if both the children are invalid, then we shall redo the crossover
  while(child1_validation==0 and child2_validation==0):
      # print(f"child1 {child1} is invalid and child2 {child2} is also invalid, regenrating the population...")
      start = random.randint(0, length - 2)
      end = random.randint(start + 1, length)

      child1 = ordered_crossover(parent1, parent2, start, end)
      child2 = ordered_crossover(parent2, parent1, start, end)

      child1_validation = overall_validation(child1, first_city, last_city, city_sequence_visit, city_sequence_avoid)
      child2_validation = overall_validation(child2, first_city, last_city, city_sequence_visit, city_sequence_avoid)

  # if either of the child is invalid, then we shall perform random mutation
  while(child1_validation==0):
    # print(f"child1 {child1} is invalid, regenrating child1...")
    random_mutation(child1)
    child1_validation = overall_validation(child1, first_city, last_city, city_sequence_visit, city_sequence_avoid)

  while(child2_validation==0):
    # print(f"child2 {child2} is invalid, regenrating child2...")
    random_mutation(child2)
    child2_validation = overall_validation(child2, first_city, last_city, city_sequence_visit, city_sequence_avoid)

  # print(f"Found valid child1 as {child1} and child2 as {child2}")
  return child1, child2

# mutated chromosome from parents
def parent_mutation(population,percentage,first_city, last_city, city_sequence_visit, city_sequence_avoid):
  '''
  This function will select 5% of the target population for mutation. It will follow an
  iterative process to generate new child. If it fails to produce any valid chromosome after
  certain number of iterations, then the process will exit
  '''
  mutation_list = random.sample(population, int(len(population)*percentage/100))
  mutated_list = []
  for chromosome in mutation_list:
    mutated_chromosome = random_mutation(chromosome)
    validation_result = overall_validation(mutated_chromosome, first_city, last_city, city_sequence_visit, city_sequence_avoid)
    while(validation_result==0):
      mutated_chromosome = random_mutation(chromosome)
      validation_result = overall_validation(mutated_chromosome, first_city, last_city, city_sequence_visit, city_sequence_avoid)
    # print("Valid Mutated Parent Found, finding next...")
    mutated_list.append(mutated_chromosome)

  return mutated_list

# Define stopping criteria

def stopping_condition(current_best_val,prev_best_val,current_iter,max_iter,value_count,max_repeat,current_population,minimum_population_size):
  '''
  The iteration will stop in either of the following cases
    1. The is no improvement in last certain number of iteration
    2. The population size is below certain threshold
    3. Iteration number reached
  '''
  stop1=0
  stop2=0
  stop3=0

  # building stop condition 1
  if current_best_val>prev_best_val:
    prev_best_val=current_best_val
    value_count=0
  else:
    value_count+=1

  if value_count>=max_repeat:
    stop1=1
    print("There is no improvement in fitness value, stopping the iteration...")
    return(stop1)
  # building stop condition 2
  if len(current_population)<minimum_population_size:
    stop2=1
    print("there is no sufficient population availble to proceed, stopping the iteration...")
    return(stop2)

  # building stop condition 3
  if current_iter>=max_iter:
    stop3=1
    print("Maximum iteration reached, stopping the iteration...")
    return(stop3)

  return min(stop1+stop2+stop3,1)


## ----------------------------- Final Run -----------------------------

In [144]:
# parameter declaration
node_number = 101
initial_population_size = 5000
minimum_population_size = 50
starting_node_id = 2
first_city=4
last_city=11
city_sequence_visit=[5,9]
city_sequence_avoid=[3,4]

# initialize data
node_ids = [i for i in range(1, node_number+1)]
time_matrix = np.random.randint(1, 100, size=(node_number, node_number))

# Runtime Parameter Initialization
overall_best_fitness = 0
overall_best_sol = []
should_stop = 0
current_iteration = 1
max_iteration=1000
prev_best_fitness=0
value_count=0
max_repeat=10
max_effort = 25
parent_mutation_percent=5

In [145]:
# initialize data
node_ids = [i for i in range(1, node_number+1)]
time_matrix = np.random.randint(1, 100, size=(node_number, node_number))
# Generate initial population
current_population = initial_population_generation(node_number, initial_population_size, starting_node_id, first_city, last_city, city_sequence_visit, city_sequence_avoid)

# calculate fitness and get maximum fitness and corresponding key
fitness_dictionary = fitness_calculation(current_population,time_matrix,starting_node_id)
current_best_sol = max(fitness_dictionary, key=fitness_dictionary.get)
current_best_fitness = fitness_dictionary[current_best_sol]

# update best value
if(current_best_fitness>overall_best_fitness):
    overall_best_fitness=current_best_fitness
    overall_best_sol=current_best_sol
should_stop=stopping_condition(current_best_fitness,prev_best_fitness,current_iteration,max_iteration,value_count,max_repeat,current_population,minimum_population_size)

# put current fitness in old fitness
prev_best_fitness=current_best_fitness

# update iteration
current_iteration += 1

solution [41, 71, 34, 98, 95, 28, 32, 45, 52, 97, 13, 91, 43, 51, 36, 20, 40, 47, 25, 64, 44, 73, 88, 57, 53, 84, 23, 101, 21, 81, 14, 26, 8, 22, 100, 10, 17, 63, 35, 24, 50, 92, 6, 82, 83, 5, 69, 70, 66, 37, 11, 68, 89, 79, 74, 31, 86, 19, 1, 27, 16, 48, 46, 76, 7, 56, 12, 39, 96, 75, 61, 78, 60, 85, 15, 62, 18, 33, 59, 94, 87, 93, 65, 29, 99, 77, 30, 38, 54, 49, 55, 42, 80, 4, 72, 3, 90, 67, 58, 9] is invalid, regenerating chromosome...
solution [47, 78, 44, 64, 45, 71, 88, 92, 93, 50, 62, 49, 21, 46, 70, 11, 81, 16, 97, 39, 4, 61, 98, 42, 57, 28, 86, 9, 95, 22, 66, 40, 41, 73, 74, 85, 34, 19, 31, 100, 87, 43, 5, 79, 77, 30, 84, 1, 10, 36, 99, 29, 32, 72, 52, 67, 27, 91, 90, 38, 58, 6, 55, 14, 37, 18, 82, 25, 69, 96, 94, 17, 3, 83, 48, 23, 33, 51, 59, 13, 60, 8, 24, 80, 56, 15, 54, 63, 65, 12, 75, 53, 35, 101, 26, 68, 20, 89, 7, 76] is invalid, regenerating chromosome...
solution [56, 69, 22, 78, 46, 48, 7, 86, 52, 38, 49, 71, 11, 98, 97, 85, 16, 75, 27, 88, 95, 63, 12, 45, 21, 72, 1

In [146]:
###--- Complete the iteration ---###

while(should_stop==0):

    print(f" Iteration number {current_iteration} started")
    # select population for next generation creation
    selected_popultion=pool_division(fitness_dictionary)
    top_10_prcnt = selected_popultion[0]
    top_85_prcnt = selected_popultion[1]

    # crossover and mutation for new generation + from previous population
    new_generation = top_10_prcnt
    remaining_population_required = initial_population_size - len(new_generation)

    # mutation of parents
    print("Mutation of Parents started")
    mutation_list = parent_mutation(top_85_prcnt,parent_mutation_percent,first_city,last_city,city_sequence_visit,city_sequence_avoid)
    print("Mutation of Parents completed")

    # crossover of parents
    print("Crossover of Parents started")
    remaining_population_required=remaining_population_required-len(mutation_list)
    crossover_population=[]

    while(len(crossover_population)<remaining_population_required):
        parent1 = random.choice(top_85_prcnt)
        parent2 = random.choice(top_85_prcnt)
        new_children = valid_child_generation(parent1,parent2, first_city, last_city, city_sequence_visit, city_sequence_avoid)
        crossover_population.extend(new_children)
        # print(f"total number of new childrens is {len(crossover_population)}")

    print("Crossover of Parents completed")

    # final new generation
    current_population=new_generation+mutation_list+crossover_population

    # calculate fitness for current pool
    fitness_dictionary = fitness_calculation(current_population, time_matrix, starting_node_id)
    current_best_sol = max(fitness_dictionary, key=fitness_dictionary.get)
    current_best_fitness = fitness_dictionary[current_best_sol]
    print(f"Current best travel time is {int(10000/fitness_dictionary[current_best_sol])} with best route as {current_best_sol}")
    # update best value
    if(current_best_fitness>overall_best_fitness):
      overall_best_fitness=current_best_fitness
      overall_best_sol=current_best_sol
      print(f"New best possible minimum travel time is {int(10000/overall_best_fitness)}")
      print(f"New best route is {overall_best_sol}")

    # Emulate value_count logic manually: track improvement stalls
    if current_best_fitness <= prev_best_fitness:
        value_count += 1
    else:
        value_count = 0
        prev_best_fitness = current_best_fitness
    should_stop=stopping_condition(current_best_fitness,prev_best_fitness,current_iteration,max_iteration,value_count,max_repeat,current_population,minimum_population_size)
    if(should_stop==1):
        print("Stopping Condition Arrived")
        break

    # update iteration
    print(f"Iteration number {current_iteration} completed")
    current_iteration += 1

print(f"Best minimum time of travel is {int(10000/overall_best_fitness)}")
print(f"Best route to travel is {overall_best_sol}")

 Iteration number 2 started
Mutation of Parents started
Mutation of Parents completed
Crossover of Parents started
Crossover of Parents completed
Current best travel time is 3787 with best route as (22, 17, 67, 48, 31, 7, 5, 51, 41, 43, 26, 13, 50, 95, 75, 25, 39, 64, 20, 37, 53, 52, 9, 76, 42, 1, 23, 79, 63, 86, 65, 91, 36, 83, 54, 71, 101, 21, 12, 6, 62, 4, 27, 58, 46, 88, 15, 98, 81, 32, 85, 59, 61, 18, 10, 57, 82, 11, 34, 73, 40, 33, 72, 47, 55, 94, 49, 3, 100, 68, 60, 78, 84, 24, 93, 89, 14, 8, 45, 77, 90, 70, 19, 97, 56, 16, 66, 99, 74, 44, 80, 92, 30, 29, 69, 28, 35, 87, 96, 38)
Iteration number 2 completed
 Iteration number 3 started
Mutation of Parents started
Mutation of Parents completed
Crossover of Parents started
Crossover of Parents completed
Current best travel time is 3787 with best route as (22, 17, 67, 48, 31, 7, 5, 51, 41, 43, 26, 13, 50, 95, 75, 25, 39, 64, 20, 37, 53, 52, 9, 76, 42, 1, 23, 79, 63, 86, 65, 91, 36, 83, 54, 71, 101, 21, 12, 6, 62, 4, 27, 58, 46, 88, 